In [1]:
from numba import cuda
import numpy as np
import math

## Data loader

In [2]:
def read_data(fn):
    with open(fn, "r") as fh:
        data = fh.readlines()
        return np.asarray([int(line.strip()) for line in data if line.strip()])

- -1 = Decreasing
- 0 = NA or stay the same
- 1 = Increasing

## Part 1

In [3]:
@cuda.jit
def increasing_decreasing(in_arr, out_arr):
    pos = cuda.grid(1)
    if pos < in_arr.size and pos - 1 >= 0:
        last = in_arr[pos - 1]
        this = in_arr[pos]
        
        # Check if increasing
        if last - this < 0:
            out_arr[pos] = 1
            
        # Check if decreasing
        if last - this > 0:
            out_arr[pos] = -1

In [4]:
def run_part1(arr):
    # Create output array
    output_arr = np.zeros_like(arr)
    
    # Configure grid
    threads = 128
    blocks = math.ceil(len(arr) / threads)
    
    # Calculate increasing/decreasing
    increasing_decreasing[blocks, threads](arr, output_arr)
    
    # Count increasing
    unique, counts = np.unique(output_arr, return_counts=True)
    c = dict(zip(unique, counts))
    
    # Return answer
    return c[1]

## Solve P1 Sample

In [5]:
sample_arr = read_data("sample.txt")
sample_arr

array([199, 200, 208, 210, 200, 207, 240, 269, 260, 263])

In [6]:
run_part1(sample_arr)

7

## Solve P1 Problem

In [7]:
input_arr = read_data("input.txt")
input_arr

array([ 157,  148,  149, ..., 8016, 8020, 8026])

In [8]:
run_part1(input_arr)

1548

## Part 2

In [9]:
@cuda.jit
def make_sums(in_arr, out_arr):
    pos = cuda.grid(1)
    if pos < in_arr.size - 2:
        out_arr[pos] = in_arr[pos] + in_arr[pos + 1] + in_arr[pos + 2] 

In [10]:
def run_part2(arr):
    # Create output array
    output_arr = np.zeros_like(arr)
    
    # Configure grid
    threads = 128
    blocks = math.ceil(len(arr) / threads)
    
    # Make the sums
    make_sums[blocks, threads](arr, output_arr)
    
    # Create next output array
    arr = output_arr
    output_arr = np.zeros(len(arr) - 2)
    
    # Calculate increasing/decreasing
    increasing_decreasing[blocks, threads](arr, output_arr)
    
    # Count increasing
    unique, counts = np.unique(output_arr, return_counts=True)
    c = dict(zip(unique, counts))
    
    # Return answer
    return c[1]

In [11]:
run_part2(sample_arr)

5

In [12]:
run_part2(input_arr)

1589